## Unit Testing

**Estimated Time To Complete This Notebook: 1 hour.**

This notebook covers **Unit Testing** with [xUnit](https://xunit.net), [FluentAssertions](https://fluentassertions.com) and [Moq](https://github.com/devlooped/moq) using [Test Driven Development (TDD)](https://en.wikipedia.org/wiki/Test-driven_development).

Microsoft's four part video series about [Unit Testing](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-test-driven-and-scenario-based-testing) with an associated [GitHub Repository](https://github.com/skimedic/presentations/tree/master/UnitTesting) is great if you haven't done unit testing before, but uses Visual Studio instead of Visual Studio Code.

We will explore unit testing in Visual Studio Code together in this notebook, using the [xUnit](https://xunit.net) unit testing framework.

- First we will create a simple `Utils` solution to explore basic unit testing concepts.
- Then we will unit test our `Todo.Api` project's `TodoItemsController`.

---

### Create a `Utils` solution

Let's create a new solution in which we can explore some basic unit testing concepts.

Execute the cell below to create a new folder `Utils` containing a new solution `Utils.sln`, and open it in a new VSCode intance.

In [1]:
%%system

mkdir ../Utils
cd ../Utils
dotnet new sln -n Utils
code .

['The template "Solution File" was created successfully.', '']

In the new VSCode isntance, right-click the file `Utils.sln` and choose `Open Solution` to show `Solution Explorer`.

---

### Add a `Utils.Maths` project to the `Utils` solution

- Right-click the `Utils` solution node in `Solution Explorer` and choose `New Project`.
- In the `Command Palette` choose:
  - Create a new .NET project: `Class Library`
  - Name the new project: `Utils.Maths`
  - Select location for the new project: `Default Directory`
  - Create project or view options: `Create project`

This will add the `Utils.Maths` project to the `Utils` solution (notice the new `Utils.Maths` project node under the `Utils` solution node in `Solution Explorer`).

Right-click the file `Class1.cs` in `Solution Explorer` and choose `Delete`.

---

### Add a simple  Calculator to the `Utils.Maths` project

- Right-click the `Utils.Maths` project node in `Solution Explorer` and choose `New File`.
- In the `Command Palette` choose:
  - Create a new file: `Interface`
  - Name the new file: `IConverter`
  - Open the file `IConverter.cs` and repalce its contents with the code below:

```csharp
using System;

namespace Utils.Maths;

public interface IConverter
{
    double Convert(decimal value);
    decimal Convert(double value);
}
```

- Right-click the `Utils.Maths` project node in `Solution Explorer` and choose `New File`.
- In the `Command Palette` choose:
  - Create a new file: `Class`
  - Name the new file: `Converter`
  - Open the file `Converter.cs` and repalce its contents with the code below:

```csharp
using System;

namespace Utils.Maths;

public class Converter : IConverter
{
    public double Convert(decimal value)
    {
        return decimal.ToDouble(value);
    }

    public decimal Convert(double value)
    {
        return (decimal)value;
    }
}
```

- Right-click the `Utils.Maths` project node in `Solution Explorer` and choose `New File`.
- In the `Command Palette` choose:
  - Create a new file: `Class`
  - Name the new file: `Calculator`
  - Open the file `Calculator.cs` and repalce its contents with the code below:

```csharp
using System;

namespace Utils.Maths;

public class Calculator
{
    private readonly IConverter _converter;

    public Calculator(IConverter converter)
    {
        _converter = converter;
    }

    public decimal Add(decimal a, decimal b)
    {
        double x = _converter.Convert(a);
        double y = _converter.Convert(b);
        double result = x + y;
        return _converter.Convert(result);
    }

    public decimal Subtract(decimal a, decimal b)
    {
        double x = _converter.Convert(a);
        double y = _converter.Convert(b);
        double result = x - y;
        return _converter.Convert(result);
    }
}
```

This adds the following to the `Utils.Maths` project:

- A `IConverter` interface with two methods to convert between a double and a decimal.
- A `Converter` class that implements `IConverter` and defines the functionality to convert between a double and a decimal.
- A `Calculator` class with two method for adding and subrtacting two decimals.
- Notice that the `Calculator` class:
  - Takes an `IConverter` as a constructor argument.
  - Uses the underlying `Converter` instance to convert between decimals and doubles in its `Add` and `Subtract` methods.

Let's use the `Calcualtor` to explore some basic unit testing concepts. 

---

### Basic Concepts in Unit Testing

In `Unit Testing`:

- We want to test (verify) the correct functionality of a class, where this class is often referred to as the `Subject Under Test (SUT)`.
  - In our example, the `Calculator` class is our `SUT`.
- The `Unit` in `Unit Testing` refers to a `unit of code`, which is often a method in a SUT.
  - In our example, the `Calculator`'s `Add()` and `Subtract()` methods are `unit of code` we want to test (verify) for correct functionality.

An important concept when designing software, is to write `testable code`, which includes:

  - Defining an interface (`IConverter`) for any dependencies (`Converter`).
  - Injecting dependencies (`Converter`) into a class (`Calculator`), via its constructor, hidden behind an interface (`IConverter`).
  - This is also known as `dependency injection`, since the dependency (`Converter`) is injected into the class (`Calculator`) via its constructor.

Often the dependencies are in the form of services that a class uses, where:

- We only want to test the `SUT`'s methods.
- We don't want to test the services' methods (since we would actually be unit testing the services then, but we only want to unit test the SUT).
- Furthermore, we want our unit test to execute quickly, and not affect any external dependencies (services).
  - If a service accesses a database, or communicates with an external RestAPI, this might take too long, and will also affect these external services.

To isolate a `SUT` from its dependencies (service):

- We create `mocks` (mockups, or fakes) of our dependencies (services), which includes `mocking` their methods.
- `Mocking` a method is simply replacing it with a mocked (fake) version, where we can:
  - Define what value the mocked method should return, when calling it with specific input parameters, which we also define.
- This is why we want all our dependencies (services) to:
  - Define an interface (it's a LOT easier to mock an interface than a concrete class).
  - Be injected into our classes, hidden behind an interface, rather than instantiating teh dependencies (services) in our class' constructor.

We want to do this ...

```csharp
public class Calculator
{
    private readonly IConverter _converter;

    public Calculator(IConverter converter) // <--- dependency injection (via the class' constructor, hidden behind an interface)
    {
        _converter = converter;
    }
}
```

... not this ...

```csharp
public class Calculator
{
    private readonly IConverter _converter;

    public Calculator()
    {
        _converter = new Converter(); // <--- instantiating the dependency inside the class
    }
}
```

Why is this advantageous?

- By using dependency injection and service interfaces above, we can:
  - Create a `mock` class implementing the service's interface.
  - `Mock` implementations of the methods in the interface, in the `mock` class.
  - Pass an instance of the `mock` class into the SUT's constructor when unit testing the SUT (hence bypassing the service entirely).

```csharp
// Mock class
public class MockConverter : IConverter
{
    public double Convert(decimal value) // <-- mock method
    {
        // return whatever value we like, when receiving whatever input parameters we like
    }

    public decimal Convert(double value) // <-- mock method
    {
        // return whatever value we like, when receiving whatever input parameters we like
    }
}

// Unit testing with a custom mock
MockConverter mockConverter = new MockConverter();
Calculator sut = new Calculator(mockConverter); // <--- inject the mock object with its mock methods into the SUT's constructor 
sut.Add();
sut.Subtract();
```

Although, as we will see, we don't have to write the mock classes ourselves, since we can use the `Moq` library to do this for use.

```csharp
// Unit testing with the Moq library
var mockConverter = Mock<IConverter>();
Calculator sut = new Calculator(mockConverter); // <--- inject the mock object with its mock methods into the SUT's constructor 
sut.Add();
sut.Subtract();
```

---

### Add a `Utils.Maths.UnitTests` Project to the `Utils` Solution

A common convention in unit testing is to:

- Create a separate unit test project for the units test.
- Reference the project with the SUT we want to test from the unit test project.
- Give the unit test project the same name as the project with the SUT, but add the suffix `Tests` or `UnitTests`.

Let's add an xUnit test project to our `Utils` solution:

- Right-click the `Utils` Solution node in `Solution Explorer` and choose `New Project`.
- In the `Command Palette` choose:
  - Create a new .NET project: `xUnit Test Project`
  - Name the new project: `Utils.Maths.UnitTests`
  - Select location for the new project: `Default Directory`
  - Create project or view options: `Create project`

This will add the `Utils.Maths.UnitTests` project to the `Utils` solution (notice the new `Utils.Maths.UnitTests` project node under the `Utils` solution node in `Solution Explorer`).

Let's also add the required NuGet packages:

- Right-click the `Utils.Maths.UnitTests` project node in `Solution Explorer`, choose `Add NuGet Package`, then make the following choices in the `Command Palette`:
  - Add NuGet Package: `Moq`
  - Select: `Moq`
  - Choose `4.20.72` (which is currently the latest version)
- Right-click the `Utils.Maths.UnitTests` project node in `Solution Explorer`, choose `Add NuGet Package`, then make the following choices in the `Command Palette`:
  - Add NuGet Package: `FluentAssertions`
  - Select: `FluentAssertions`
  - Choose `4.14.0` (which is currently the latest **compatible** version)

We also need a reference to the `Utils.Maths` project:

- Right-click the `Utils.Maths.UnitTests` project node in `Solution Explorer`, choose `Add Project Reference`, and select `Utils.Maths`.

Finally, remove the file `UnitTest1.cs` in the `Utils.Maths.UnitTests` project:

- Right-click the file `UnitTest1.cs` in `Solution Explorer` and choose `Delete`.

---

### Add a Test Class `CalculatorTests` with Test Methods

A common convention in unit testing is to give a test class the same name as the SUT it's testing, but with the suffix `Tests`:

- Right-click the `Utils.Maths.UnitTests` project in `Solution Explorer`, choose `New File` and the file type to `Class`, and name it `CalculatorTests`.
- Open the file `CalculatorTests.cs` and replace its contents with the code below:

```csharp
using System;

namespace Utils.Maths.UnitTests;

public class CalculatorTests
{
    public CalculatorTests()
    {

    }

    [Fact]
    public void Add_TwoNumbers_ReturnsCorrectSum()
    {
        // Arrage
        decimal a = 1;
        decimal b = 2;
        decimal expected = 3;
        Converter converter = new Converter();
        Calculator calculator = new Calculator(converter); // <-- SUT

        // Act
        decimal actual = calculator.Add(a, b);
        
        // Assert
        Assert.Equal(expected, actual);
    }

    [Fact]
    public void Subtract_TwoNumbers_ReturnsCorrectDifference()
    {
        // Arrage
        decimal a = 1;
        decimal b = 2;
        decimal expected = -1;
        Converter converter = new Converter();
        Calculator calculator = new Calculator(converter); // <-- SUT

        // Act
        decimal actual = calculator.Subtract(a, b);
        
        // Assert
        Assert.Equal(expected, actual);
    }
}
```

Let's examine the structure of unit tests with the `xUnit` framework:

- The name of the test class `CalculatorTests` has the same name as the SUT class `Calculator`, but with the suffix `Tests`.
- The test class has a constructor, where we can choose to define a so called `Test Fixure` (a setup that is run before each test).
- One or more Test Methods, each:
  - Adorned with the attribute (decorator) `[Fact]`, which tells `xUnit` that this is a test method (so it can find it when scanning for tests).
  - With a name based on the naming convention `<MethodName>_<StateUnderTest>_<ExpectedBehavior>` where:
  - `<MethodName>` is the same name as the SUT's method we are testing (e.g. `Add`).
  - `<StateUnderTest>` is the context, or state, we are testing (e.g. `TwoNumbers` or `TwoDecimals` or `TwoPositiveDecimals`, etc.).
  - `<ExpectedBehavior>` is the expected behavior, or result, of the test (e.g. `ReturnsCorrectSum` or `ThrowsException`, etc.).
- Each test method has a method body with three distinct sections `Arrange`, `Act` and `Assert`, where:
  - `Arrange` is used to set up everything that is needed for the test, including creating an instance of the SUT class.
  - `Act` is where the test is executed (usually one line of code calling one method in the SUT).
  - `Assert` is where we assefrt (verify) the actual result is the same as the expected result.
- Note: part of the `Arrange` set up can be done in a so called `Test Fixture` inside the Test Class' Constructor:
  - For example, if every Test Method's `Arrange` section does exactly the same thing over a number of rows, these can be moved into the Constructor.
  - The Constructor is invoked before EACH Test Method is run (i.e. once for EACH Test Method).
- More advanced `Test Fixture`s can be set up (see [The xUnit Documentation](https://xunit.net)).


```csharp
public class CalculatorTests // <--- Test Class (namving convention: <NameOfSUT>Tests)
{
    public CalculatorTests() // <--- Test Class Constructor
    {
        // Test Fixture
    }

    [Fact]
    public void Add_TwoNumbers_ReturnsCorrectSum() // <-- Test Method (naming convention: <MethodName>_<StateUnderTest>_<ExpectedBehavior>)
    {
        // Arrage       // <-- Set up everything that is needed for the test
        
        // Act          // <-- Execute the test
        
        // Assert       // <-- Assert (verify) the actual result is the same as the expected result
    }
}
```

`xUnit` also has a utility, called `ITestOutputHelper`, we can use for printing output during tests:

- We inject `ITestOutputHelper` via the Test Class Constructor, and then use its `WriteLine()` method.

Here's an example of moving part of the `Arrange` section into the Constructor, and using `ITestOutputHelper`.

- Open the file `CalculatorTests.cs` and replace its contents with the code below:

```csharp
using System;
using Xunit.Abstractions;

namespace Utils.Maths.UnitTests;

public class CalculatorTests
{
    private decimal a;
    private decimal b;
    private Converter converter;
    private Calculator calculator; // <-- SUT
    private readonly ITestOutputHelper _output;

    public CalculatorTests(ITestOutputHelper output)
    {
      _output = output;

        // Test Fixture
        a = 1;
        b = 2;
        converter = new Converter();
        calculator = new Calculator(converter); // <-- SUT
    }

    [Fact]
    public void Add_TwoNumbers_ReturnsCorrectSum()
    {
        // Arrage
        decimal expected = 3;

        // Act
        decimal actual = calculator.Add(a, b);
        _output.WriteLine($"{a} + {b} = {actual}");
        
        // Assert
        Assert.Equal(expected, actual);
    }

    [Fact]
    public void Subtract_TwoNumbers_ReturnsCorrectDifference()
    {
        // Arrage
        decimal expected = -1;

        // Act
        decimal actual = calculator.Subtract(a, b);
        _output.WriteLine($"{a} - {b} = {actual}");
        
        // Assert
        Assert.Equal(expected, actual);
    }
}
```

---

### Running Test in VSCode, and Using the Test Explorer

Now that we have added unit tests for our `Calculator`, let's see now we can run the tests in VSCode.

<style>
    .container {
        width: 98%;
        margin-left: 0; /* Push the container to the left */
        margin-right: auto; 
    }
    .text-image {
        margin-bottom: 35px; /* Space between sections */
        overflow: hidden; /* Ensure image stays within the container */
    }
    .text {
        text-align: justify; /* Justify the text for better readability */
    }
    .image {
        float: right; /* Float the image to the right */
        margin-left: 25px; /* Space between image and text */
        margin-bottom: 10px; /* Space between image and text */
        max-width: 50%; /* Limit image size */
        height: auto; /* Maintain aspect ratio */
    }
</style>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/test-explorer-calculator.png" />
        <div class="text">
            <p>
                Build the <b>Utils.Maths.UnitTests</b> project by right-clicking the <b>Utils.Maths.UnitTests</b> node in <b>Solution Explorer</b> and choosing <b>Build</b>.
            </p>
            <p>
                Reload the VSCode window so that Test Explorer can find the tests (this is only required once, and is a current bug in VSCode), by opening the <b>Command Palette</b> with <b>Ctrl + Shift + P</b> (<b>Cmd + Shift + P</b> on a Mac), or via the main menu <b>View -> Command Palette</b>, and choosing <b>Developer: Reload Window</b>.
            </p>
            <p>
              Click the testing icon <img src="../images/test-explorer-view-icon.png"/> in the <b>Activity Bar</b>.
            </p>
            <p>
              This displays the <b>TEST EXPLORER</b> in the <b>Primary Side Bar</b>.
            </p>
            <p>
              Click the run tests icon <img src="../images/run-tests-icon.png"/>, to run all tests (<b>xUnit</b> will find all methods with the <b>[Fact]</b> attribute in the currenct solution and run them).
              <ul>
                <li>If test has a green icon next to it, the test passed successfully (i.e. all <b>Assert</b>s returned <b>True</b>).</li>
                <li>If the test has a red icon next to it, the test failed (i.e. at least one <b>Assert</b> returned <b>False</b>).</li>
              </ul>
            </p>
            <p>
                You can also click the green (red) icon in the code editor's left margin to execute a test.
            </p>
            <p>
                To debug a test, you can add a breakpoint in the code editor's left margin, and then click on the debug tests icon <img src="../images/debug-tests-icon.png" /> in the <b>TEST EXPLORER</b>.
            </p>
        </div>
    </div>
    <div class="text-image">
        <img class="image" src="../images/test-results-calculator.png" />
        <div class="text">
            <p>
                The test results are also shown in the <b>TEST RESULTS</b> tab, under the code editor.
            </p>
            <p>
                If you used the <code>ITestOutputHelper</code>, the output from each <code>WriteLine()</code> is also shown here.
            </p>
        </div>
    </div>
</div>

---

### Adding Mocks with `Moq`

Currently, we haven't mocked our `Calculator` SUT's dependencies (services).

Let's see how we can do this using the `Moq` library.

- Open the file `CalculatorTests.cs` and replace its contents with the code below:

```csharp
using System;
using Moq;
using Xunit.Abstractions;

namespace Utils.Maths.UnitTests;

public class CalculatorTests
{
    private readonly ITestOutputHelper _output;

    public CalculatorTests(ITestOutputHelper output)
    {
        _output = output;
    }

    [Fact]
    public void Add_TwoNumbers_ReturnsCorrectSum()
    {
        // Arrage
        decimal a = 1;
        decimal b = 2;
        decimal expected = 3;
        
        var mock = new Mock<IConverter>();                   // <--- Here we are creating a mock of the IConverter interface

        mock.Setup(m => m.Convert(a)).Returns(1.0);          // <--- Here we defining what the Convert() ...
        mock.Setup(m => m.Convert(b)).Returns(2.0);          // <--- ... method in IConverter should return ...
        mock.Setup(m => m.Convert(3.0)).Returns(expected);   // <--- ... when receiving specific parameters
        
        Calculator calculator = new Calculator(mock.Object); // <--- Here we pass in an IConverter mock Object as an argument to the SUT's constructor

        // Act
        decimal actual = calculator.Add(a, b);
        _output.WriteLine($"{a} + {b} = {actual}");
        
        // Assert
        Assert.Equal(expected, actual);
    }

    [Fact]
    public void Subtract_TwoNumbers_ReturnsCorrectDifference()
    {
        // Arrage
        decimal a = 1;
        decimal b = 2;
        decimal expected = -1;

        var mock = new Mock<IConverter>();                   // <--- Here we are creating a mock of the IConverter interface

        mock.Setup(m => m.Convert(a)).Returns(1.0);          // <--- Here we defining what the Convert() ...
        mock.Setup(m => m.Convert(b)).Returns(2.0);          // <--- ... method in IConverter should return ...
        mock.Setup(m => m.Convert(-1.0)).Returns(expected);  // <--- ... when receiving specific parameters

        Calculator calculator = new Calculator(mock.Object); // <--- Here we pass in an IConverter mock Object as an argument to the SUT's constructor

        // Act
        decimal actual = calculator.Subtract(a, b);
        _output.WriteLine($"{a} - {b} = {actual}");

        // Assert
        Assert.Equal(expected, actual);
    }
}
```

Now, run the tests again and verify everything is still working.

Notice the following:

- `var mock = new Mock<IConverter>();` creates a mock of the `IConverter` interface.
- `mock.Setup(m => m.Method(parmeters)).Returns(returnvalue);` defines what the `Method` should return as its `returnvalue` when receiving `parameters` as input.
- `mock.Object` creates and returns a mock `Object` (an instance of a mock class implementing `IConverter`), with the mock methods defined by `mock.Setup()`.
- This means, that when the SUT (`Calculator`) calls these mocked methods on the `IConverter` interface, the mock methods are invoked instead of the actual `Convert` methods.
  - Hence, bypassing the implementation defined by the `Converter`'s actual methods.
- This mocking technique becomes important when working with real dependencies (services) that e.g. access databases och call external Rest APIs.
  - We need to mock these dependencies (services), so that our mocked methods are called instead, hence avoiding accessing e.g. an actual database, etc.

The `Moq` library is quite extensive, e.g.:
- `var mock = new Mock<IMyInterface>();`
  - Creates a mock of the a specific interface (in this case the interface `IMyInterface`).
- `mock.Setup(m => m.MyMethod(myArguments)).Returns(myReturnValue);`
  - Defines what a specific method (here `MyMethod`) in the mocked interface should return (here `myReturnValue`) when called with specific arguments (here `myArguments`).
- `mock.SetupSequence(m => m.MyMethod(It.IsAny<MyType>())).Returns(myReturnValue1).Returns(myReturnValue2);`
  - Defines what a specific method (here `MyMethod`) in the mocked interface should return (here `myReturnValue1` (1st call), `myReturnValue2` (2nd call)) when called with an argument of a specific type (here `It.IsAny<MyType>()`).
  - An alternative is `It.Is<int>(i => i > 0)` where the argument must be of type `int` and bigger than `0`.
  - An alternative is `It.IsInRange(0,100,Range.Inclusive)` where the argument must be within the range `0-100`.
- `var sut = new SUT(mock.Object);` creates an instance of the **Subject Under Test (SUT)**, i.e. a class (here `SUT`), but passes in the mock object (`mock.Object`) in place of the interface (e.g.`IMyInterface`) it is mocking.
- `var call = m => m.MyMethod(myArguments);` followed by `mock.Setup(call).Returns(myReturnValue).Verifyable(MyErrorMessage);` makes the method call (here `call`) *verifiable*.
  - After the SUT has been called, the called method `call` can be verified with e.g. the methods below, and if they aren't met, the error message (here `MyErrorMessage`) is shown (and the test fails):
    - `mock.Verify(call)` verifies the method `call` was called.
    - `mock.Verify(call, Times.Once)` verifies the method `call` was called exactly `1` time.
    - `mock.Verify(call, Times.Exactly(2))` verifies the method `call` was called exactly `2` times.
    - `mock.Verify(call, Times.Never)` verifies the method `call` was `never` called.
- `mock.Setup(m => m.MyMethod(myArguments)).Throws<MyException>();`
  - Defines what a specific method (here `MyMethod`) in the mocked interface should throw an exception (here `MyException`) when it is called with specific arguments (here `myArguments`).
- To learn more, see the [Moq Quickstart](https://github.com/devlooped/moq/wiki/Quickstart).

---

### The `[Theory]` and `[InlineData]` Attributes

It would be handy to run the Test Methods with multiple sets of data.

If we use the `[Fact]` attribute, we would have to write a Test Method for each set of data.

Instead, we can replace the `[Fact]` attribute with the `[Theory]` and one `[InlineData]` attribute for each set of data.

To see how this works:

- Right-click the `Utils.Maths.UnitTests` project node in `Solution Explorer` and choose `New File`.
  - In the `Command Palette`, choose the file type to `Class`, and name it `CalculatorInlineDataTests`.
- Open the file `CalculatorInlineDataTests.cs` and replace its contents with the code below;

```csharp
using System;
using Moq;
using Xunit.Abstractions;

namespace Utils.Maths.UnitTests;

public class CalculatorInlineDataTests
{
    private readonly ITestOutputHelper _output;

    public CalculatorInlineDataTests(ITestOutputHelper output)
    {
        _output = output;
    }

    [Theory] // <---------------------- We have replaced the [Fact] attribute with the [Theory] attribute 
    [InlineData(1, 2, 3)] // <--------- and two [InlineData] attributes, each with three values
    [InlineData(2, 4, 6)] // <--------- that are sent in as input parameter to the Test Method ---------------\
    public void Add_TwoNumbers_ReturnsCorrectSum(decimal a, decimal b, decimal expected) // <-----------------/
    {
        // Arrage
        
        var mock = new Mock<IConverter>();
        mock.Setup(m => m.Convert(a)).Returns(1.0);
        mock.Setup(m => m.Convert(b)).Returns(2.0);
        mock.Setup(m => m.Convert(3.0)).Returns(expected);
        
        Calculator calculator = new Calculator(mock.Object);

        // Act
        decimal actual = calculator.Add(a, b);
        _output.WriteLine($"{a} + {b} = {actual}");
        
        // Assert
        Assert.Equal(expected, actual);
    }

    [Theory]// <---------------------- We have replaced the [Fact] attribute with the [Theory] attribute
    [InlineData(1, 2, -1)] // <--------- and two [InlineData] attributes, each with three values
    [InlineData(2, 4, -2)] // <--------- that are sent in as input parameter to the Test Method --------------\
    public void Subtract_TwoNumbers_ReturnsCorrectDifference(decimal a, decimal b, decimal expected) // <-----/
    {
        // Arrage

        var mock = new Mock<IConverter>();
        mock.Setup(m => m.Convert(a)).Returns(1.0);
        mock.Setup(m => m.Convert(b)).Returns(2.0);
        mock.Setup(m => m.Convert(-1.0)).Returns(expected);

        Calculator calculator = new Calculator(mock.Object);

        // Act
        decimal actual = calculator.Subtract(a, b);
        _output.WriteLine($"{a} - {b} = {actual}");

        // Assert
        Assert.Equal(expected, actual);
    }
}
```

Notice, that when we use the `[Fact]` attribute, we can only use one set of data (e.g. `a=1, b=2, expected=3`) inside the Test Method.

When we use the `[Theory]` attribute, we can supply as many sets of data as we like, each within its own `[InlineData]` attribute.

- Each `[InlineData]` attribute has a number of values within the parantheses, e.g. `[InlineData(1,2,3)]`.
- For each `[InlineData]`attribute, the Test Method will be called with the values within the the parantheses.
- The values within the parantheses are mapped to the Test Method's input variables (`decimal a, decimal b, decimal expected`).
- This permits us to test the same Test Method with mulitple sets of data.

Run the tests again to verify everything is working as expected.

---

### The `[Theory]` and `[MemberData]` Attributes

If we have a lot of test data, we can replace the numerous `[InlineData]` attributes with one `[MemberData]` attribute.

- The `[MemberData]` attribute takes the name of a static method within its parentheses, where the static method returns a set of data.

To see how this works:

- Right-click the `Utils.Maths.UnitTests` project node in `Solution Explorer` and choose `New File`.
  - In the `Command Palette`, choose the file type to `Class`, and name it `CalculatorMemberDataTests`.
- Open the file `CalculatorMemberDataTests.cs` and replace its contents with the code below;


```csharp
using System;
using Moq;
using Xunit.Abstractions;

namespace Utils.Maths.UnitTests;

public class CalculatorMemberDataTests
{
    private readonly ITestOutputHelper _output;

    public CalculatorMemberDataTests(ITestOutputHelper output)
    {
        _output = output;
    }

    public static IEnumerable<object[]> AdditionData => new List<object[]> // <--- A static method that returns an array of data
    {
        new object[] { 1, 2, 3 },
        new object[] { 2, 4, 6 }
    };

    public static IEnumerable<object[]> SubtractionData => new List<object[]> // <--- A static method that returns an array of data
    {
        new object[] { 1, 2, -1 },
        new object[] { 2, 4, -2 }
    };

    [Theory] // <------------------------------ We're still using the [Theory] attribute,
    [MemberData(nameof(AdditionData))] // <--- but have replaced the numerous [InlineData] attributes with one [MemberData] attribute
    public void Add_TwoNumbers_ReturnsCorrectSum(decimal a, decimal b, decimal expected)
    {
        // Arrage
        
        var mock = new Mock<IConverter>();
        mock.Setup(m => m.Convert(a)).Returns(1.0);
        mock.Setup(m => m.Convert(b)).Returns(2.0);
        mock.Setup(m => m.Convert(3.0)).Returns(expected);
        
        Calculator calculator = new Calculator(mock.Object);

        // Act
        decimal actual = calculator.Add(a, b);
        _output.WriteLine($"{a} + {b} = {actual}");
        
        // Assert
        Assert.Equal(expected, actual);
    }

    [Theory] // <-------------------------------- We're still using the [Theory] attribute,
    [MemberData(nameof(SubtractionData))] // <--- but have replaced the numerous [InlineData] attributes with one [MemberData] attribute
    public void Subtract_TwoNumbers_ReturnsCorrectDifference(decimal a, decimal b, decimal expected)
    {
        // Arrage

        var mock = new Mock<IConverter>();
        mock.Setup(m => m.Convert(a)).Returns(1.0);
        mock.Setup(m => m.Convert(b)).Returns(2.0);
        mock.Setup(m => m.Convert(-1.0)).Returns(expected);

        Calculator calculator = new Calculator(mock.Object);

        // Act
        decimal actual = calculator.Subtract(a, b);
        _output.WriteLine($"{a} - {b} = {actual}");

        // Assert
        Assert.Equal(expected, actual);
    }
}
```

When we use the `[InlineData]` attribute, we need to adorn the Test Method with one such attribute per test set.

Wen we use the `[MemberData]` attribute, we only need one such attribute:

- The `[MemberData]` attribute references a static method within its parantheses, e.g. `[MemberData(nameof(AdditionData))]`.
- The static method returns a set of data (as much as we like), e.g. a list of objects, each containing e.g. three values.

Run the tests again to verify everything is working as expected.

---

### FluentAssertions

The static `Assert` class in `xUnit` has numerous static methods for performing various kinds of assertions, e.g.:

- `Assert.Same(expected, actual)` asserts that `expected` and `actual` are the same object (have identical references, i.e. are referring to the same object).
- `Assert.Equal(expected, actual)` asserts that `expected` and `actual` are equal (have identical values).
- `Assert.Null(actual)` asserts that `actual` is `null`.
- `Assert.Throws<MyException>(() => throw new MyException())` throws a specific exception (in this case the exception `MyException`).
- `Record.Exception(() => throw new MyException())` is equivalent to `Assert.Throws<MyException>(() => throw new MyException())`.
- For more examples, see [xUnit Assertions](https://textbooks.cs.ksu.edu/cis400/1-object-orientation/04-testing/05-xunit-assertions), the [xUnit Documentation](https://xunit.net), or watch the [Microsoft's Unit Testing Video Series](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-test-driven-and-scenario-based-testing) with an associated [GitHub Repository](https://github.com/skimedic/presentations/tree/master/UnitTesting).

The static `Assert` class in `xUnit` is great, but most developer's use `FluentAssertions` instead, since they enable you to express assertions in a more natural way, e.g.:
- Instead of writing `Assert.Equal(expected, actual)`, we can write `actual.Should().Be(expected)`.
- I.e. using `FluentAssertions` expresses assertion in a more natural (fluent) way.
- The [FluentAssertions Documentation](https://fluentassertions.com/introduction) is also much better than `xUnit`'s `Assert` documentation.

We will see how we can use `FluentAssertions` when unit testing our `Todo.Api` project's `TodoItemsController`.

- We are done with the `Utils` solution, so you can close that Visual Studio Code instance.
- Now we will continue working with our `Todo` solution, so focus on that Visual Studio Code instance.

---

### Add the `Todo.Api.UnitTests` Project to the `Todo` Solution via `Solution Explorer`

Now that we have covered the basic of Unit Testing, let's unit test the `TodoItemsController` in the `Todo.Api` project.

Let's add a xUnit test project to our `Todo` solution:

- Right-click the `Todo` Solution node in `Solution Explorer` and choose `New Project`.
- In the `Command Palette` choose:
  - Create a new .NET project: `xUnit Test Project`
  - Name the new project: `Todo.Api.UnitTests`
  - Select location for the new project: `Default Directory`
  - Create project or view options: `Create project`

This will add the `Todo.Api.UnitTests` project to the `Todo` solution (notice the new `Todo.Api.UnitTests` project node under the `Todo` solution node in `Solution Explorer`).

Let's also add the required NuGet packages:

- Right-click the `Todo.Api.UnitTests` project node in `Solution Explorer`, choose `Add NuGet Package`, then make the following choices in the `Command Palette`:
  - Add NuGet Package: `FluentAssertions`
  - Select: `FluentAssertions`
  - Choose `4.14.0` (which is currently the latest **compatible** version)
- Right-click the `Todo.Api.UnitTests` project node in `Solution Explorer`, choose `Add NuGet Package`, then make the following choices in the `Command Palette`:
  - Add NuGet Package: `Moq`
  - Select: `Moq`
  - Choose `4.20.72` (which is currently the latest version)
- Right-click the `Todo.Api.UnitTests` project node in `Solution Explorer`, choose `Add NuGet Package`, then make the following choices in the `Command Palette`:
  - Add NuGet Package: `Moq.EntityFrameworkCore`
  - Select: `Moq.EntityFrameworkCore`
  - Choose `2.2.1` (which is currently the latest version)

We also need a reference to the `Todo.Api` project:

- Right-click the `Todo.Api.UnitTests` project node in `Solution Explorer`, choose `Add Project Reference`, and select `Todo.Api`.

Finally, remove the file `UnitTest1.cs` in the `Todo.Api.UnitTests` project:

- Right-click the file `UnitTest1.cs` in `Solution Explorer` and choose `Delete`.

**Note**

- The NuGet package `Moq.EntityFrameworkCore` is useful when mocking `EntityFrameworkCore`.
  - `DbContext` and `DbSet<>` don't expose an interface, so it's tricky to mock these concrete classes.
  - If you want to mock a `DbContext`, use `Moq.EntityFrameworkCore` and do `Mock<ApplicationDbContext>` (assuming `ApplicationDbContext` is a `DbContext`).
  - If you want to mock a `DbSet<>`, use `Moq.EntityFrameworkCore` and do `Mock<TodoItems>` (assuming `TodoItems` is a `DbSet<TodoItem>`).

---

### Add a class `TodoItemsControllerTests` to the `Todo.Api.UnitTests` project

Let's add a `TodoItemsControllerTest` test class.

- Right-click the `Todo.Api.UnitTests` project in `Solution Explorer`, choose `New File` and the file type to `Class`, and name the class `TodoItemsControllerTests`.
- Open the file `TodoItemsControllerTests.cs` and replace its contents with the code below.

```csharp
namespace Todo.APi.UnitTests;

public class TodoItemsControllerTests
{
    public TodoItemsControllerTests()
    {
    }

    [Fact]
    public void Update_ShouldUpdateTodoItem_WhenFound()
    {
        // Arrange

        // Act

        // Assert
    }
}
```

This is the basic structure of a `xUnit` test, where we see that we want to test the `Update()` method in the `TodoItemsController` SUT.

Let's analyze the `TodoItemsController` and its `Update()` method before completing the test.

---

### Analyzing the `TodoItemsController` and its `Update()` method

Below we see part of the code for the `TodoItemsController`, including its constructor and its `Update()` method.

```csharp
[ApiController]
[Route("api/[controller]")]
public class TodoItemsController : ControllerBase
{
    private readonly IUnitOfWork _unitOfWork; // <--- A service we need to mock
    private readonly IMapper _mapper;         // <--- A service we need to mock

    public TodoItemsController(IUnitOfWork unitOfWork, IMapper mapper) // <--- Services hidden behind interfaces and injected via the constructor
    {
        _unitOfWork = unitOfWork;
        _mapper = mapper;
    }

    [HttpPut("{id}")]
    public async Task<IActionResult> Update(int id, [FromBody] TodoItemRequestDto dto) // <--- Update() takes an int and a TodoItemsRequestDto, returning a Task<IActionResult>
    {
        TodoItem item;
        TodoItem? existingItem;
        try
        {
            item = _mapper.Map<TodoItem>(dto); // <--- The IMapper's Map<TodoItem>() method called with a TodoItemRequestDto, returning a TodoItem

            if (item == null || !ModelState.IsValid)
            {
                return BadRequest("TodoItemNameAndNotesRequired"); // <--- Task<IActionResult>'s Result property could hold a BadRequestObjectResult
            }

            existingItem = await _unitOfWork.TodoItems.FirstOrDefaultAsync(t => t.Id == id); // <--- IUnitOfWork's TodoItems property read, calling its FirstOrDefaultAsync() method
            if (existingItem == null)
            {
                return NotFound("RecordNotFound"); // <--- Task<IActionResult>'s Result property could hold a NotFoundObjectResult
            }

            existingItem.Name = item.Name;
            existingItem.Notes = item.Notes;
            existingItem.Completed = item.Completed;
            
            _unitOfWork.TodoItems.Update(existingItem); // <--- IUnitOfWork's TodoItems property read, calling its Update() method
            await _unitOfWork.CompleteAsync(); // <--- IUnitOfWork's CompleteAsync() method called
        }
        catch (Exception)
        {
            return BadRequest("CouldNotUpdateItem"); // <--- Task<IActionResult>'s Result property could hold a BadRequestObjectResult
        }
        //return NoContent();
        return Ok(_mapper.Map<TodoItemResponseDto>(existingItem)); // <--- IMapper's Map<TodoItem>() method called with a TodoItem, returning a TodoItemResponseDto
    }                                                              // <--- Task<IActionResult>'s Result property could hold a OkObjectResult
}                                                                  // <--- OkObjectResult Value proeprty holds a TodoItemResponseDto
```

Assuming we want to test the `Happy Path` through the `Update()` method, we see that:

- We need to mock the `IMapper` interface.
  - The following members are used in the `IMapper` interface:
    - `Map<TodoItem>(dto)` method, taking a `TodoItemRequestDto`, returning a `TodoItem`.
    - `Map<TodoItemResponseDto>(existingItem)` method, taking a `TodoItem`, returning a `TodoItemResponseDto`.
- We need to mock the `IUnitOfWork` interface.
  - The following members are used in the `IUnitOfWork` interface:
    - `_unitOfWork.TodoItems` property, which is a `ITodoItemRepository`.
    - `_unitOfWork.CompleteAsync()` method, returing a `Task<int>`.
- We need to mock the `ITodoItemRepository` interface.
  - The following members are used in the `ITodoItemRepository` interface:
    - `FirstOrDefaultAsync(t => t.Id == id)` taking a `Expression<Func<TodoItem,bool>>`, returning a `Task<TodoItem?>`.
    - `Update(existingItem)` taking a `TodoItem`.
- The `TodoItemsController`'s `Update()` method takes an `int` and a `TodoItemRequestDto`, returning a `Task<IActionResult>`.
  - The `Task`'s `Result` property is a `OkObjectResult` (happy path).
    - The `OkObjectResult`'s `Value` property is a `TodoItemResponseDto`.

Now that we have traced call to the `TodoItemsController`'s `Update()` method, let's unit test it using the necessry mocks. 

---

### Unit Testing the `TodoItemsController`'s `Update()` Method with `Moq` and `FluentAssertions`

Let's modify the `TodoItemsControllerTest` class.

- Open the file `TodoItemsControllerTests.cs` and replace its contents with the code below.

```csharp
using FluentAssertions;
using Todo.Dto;
using Todo.Data.Repositories;
using AutoMapper;
using Todo.Api.Controllers;
using Moq;
using Microsoft.AspNetCore.Mvc;
using Todo.Data.Entities;
using Xunit.Abstractions;

namespace Todo.APi.UnitTests;

public class TodoItemsControllerTests
{
    private readonly TodoItem _item1;
    private readonly Mock<ITodoItemRepository> _mockTodoItemRepository;
    private readonly Mock<IUnitOfWork> _mockUnitOfWork;
    private readonly Mock<IMapper> _mockMapper;
    private readonly TodoItemsController _todoItemsController;
    private readonly ITestOutputHelper _output;

    public TodoItemsControllerTests(ITestOutputHelper output)
    {
        _output = output;

        // Test Fixture

        // Sample data
        _item1 = new TodoItem { Id = 1, Name = "Learn xUnit", Notes = "Explore xUnit", Completed = false, Comments = "xUnit" };
        int Id = _item1.Id;

        // Mock TodoItemRepository 
        _mockTodoItemRepository = new Mock<ITodoItemRepository>();
        _mockTodoItemRepository.Setup(c => c.FirstOrDefaultAsync(t => t.Id == Id)).ReturnsAsync(() => _item1);
        _mockTodoItemRepository.Setup(c => c.Update(It.IsAny<TodoItem>())); // if we did this in the "Arrage" section we could pass in "todoItemOut" to Update()

        // Mock UnitOfWork
        _mockUnitOfWork = new Mock<IUnitOfWork>();
        _mockUnitOfWork.Setup(c => c.TodoItems).Returns(_mockTodoItemRepository.Object);
        _mockUnitOfWork.Setup(c => c.CompleteAsync()).ReturnsAsync(() => It.IsAny<int>()); // we could also pass in "() => 1" to ReturnsAsync()

        // Mock Mapper
        _mockMapper = new Mock<IMapper>();

        // Subject Under Test (SUT)
        _todoItemsController = new TodoItemsController(_mockUnitOfWork.Object, _mockMapper.Object);
    }

    [Fact]
    public async Task Update_ShouldUpdateTodoItem_WhenFound()
    {
        // Arrange
        var request = new TodoItemRequestDto { Name = "Learn nUnit", Notes = "Explore nUnit", Done = true };
        var todoItemIn = new TodoItem { Id = 0, Name = request.Name, Notes = request.Notes, Completed = request.Done, Comments = "" };
        var todoItemOut = new TodoItem { Id = _item1.Id, Name = request.Name, Notes = request.Notes, Completed = request.Done, Comments = _item1.Comments };
        var expected = new TodoItemResponseDto { Id = todoItemOut.Id, Name = todoItemOut.Name, Notes = todoItemOut.Notes, Done = todoItemOut.Completed };

        _mockMapper.Setup(m => m.Map<TodoItem>(request)).Returns(todoItemIn);
        _mockMapper.Setup(m => m.Map<TodoItemResponseDto>(It.IsAny<TodoItem>())).Returns(expected); // todoItemOut

        // Act
        IActionResult result = await _todoItemsController.Update(_item1.Id, request);

        // Assert

        // Verify correct status code 
        OkObjectResult ok = (OkObjectResult)result;
        ok.StatusCode.Should().Be(200);

        // Verify correct response
        TodoItemResponseDto actual = (TodoItemResponseDto)ok.Value!;
        actual.Id.Should().Be(expected.Id);
        actual.Name.Should().Be(expected.Name);
        actual.Notes.Should().Be(expected.Notes);
        actual.Done.Should().Be(expected.Done);
        _output.WriteLine($"Update({_item1.Id}): Id={actual.Id}, Name={actual.Name}, Notes={actual.Notes}, Done={actual.Done}");

        // Verify UnitOfWork.TodoItems.Add() and UnitOfWork.CompleteAsync() were called exactly once
        _mockUnitOfWork.Verify(c => c.TodoItems.Update(It.IsAny<TodoItem>()), Times.Once());
        _mockUnitOfWork.Verify(c => c.CompleteAsync(), Times.Once());
    }
}
```

Let's break down this code to see what it is doing.

- In `TodoItemsController`s constructor, we are setting up a test fixture (handy when mocking the rest of `TodoItemsController`s methods):
  - First we create some test data `_item1` (this is a `TodoItem`) and store its Id in a variable `int Id = _item1.Id`.
  - Then we mock the `ITodoItemRepository` interface (`_mockTodoItemRepository = new Mock<ITodoItemRepository>()`):
    - We setup `FirstOrDefaultAsync()` to return `_item1` when passing in `t => t.Id == Id`.
    - We setup `Update()` to take any `TodoItem`, i.e. `Update(It.IsAny<TodoItem>())` (this isn't really necessary).
  - Then we mock the `IUnitOfWork` interface (`_mockUnitOfWork = new Mock<IUnitOfWork>()`):
    - We setup `TodoItems` (property) to return the mocked TodoItemsRepository Object `_mockTodoItemRepository.Object`.
    - We setup `CompleteAsync()` to return any `int`, i.e. `() => It.IsAny<int>()` (this isn't really necessary).
  - Then we mock the `IMapper` interface (`_mockMapper = new Mock<IMapper>()`):
    - We leave the setups to the `Arrage` section in the test method `Update_ShouldUpdateTodoItem_WhenFound()`.
  - Finally, we create an instance of our SUT, passing in the mock Objects for our mock UnitOfWork and Mapper:
    -  `_todoItemsController = new TodoItemsController(_mockUnitOfWork.Object, _mockMapper.Object)`
- In `TodoItemsController`s test method `Update_ShouldUpdateTodoItem_WhenFound()`, we fill in the `Arrange`, `Act` and `Assert` sections:
  - In the `Arrage` section:
    - We create the `TodoItemRequestDto`, `TodoItemResponseDto` and two `TodoItem` instances for our Mapper.
    - We setup `Map<TodoItem>()` to return one of the `TodoItem`'s when the `TodoItemRequestDto` is passed in.
    - We setup `Map<TodoItem>()` to return the `TodoItemResponseDto`'s when any `TodoItem` is passed in (i.e. `It.IsAny<TodoItem>()`).
  - In the `Act` section:
    - We call out SUT's `Update()` method with `_item1.Id` and the `TodoItemRequestDto` as parameters, and store the return value in a variable. 
  - In the `Assert` section:
    - We assert the `OkObjectResult` is returned by the SUT's `Update()` method.
    - We extract the `TodoItemResponseDto` from the `OkObjectResult`.
    - Then we assert that all the property values in our actual and expected `TodoItemResponseDto`s are the same.
    - Finally, we assert that:
      - The mocked `TodoItemRepository`'s `Update()` method was called exactly once with any `TodoItem` as input.
      - The mocked `UnitOfWork`'s `CompleteAsync()` method was called exactly once.

---

### Run the tests

Finally we can run the tests.

- Right-click the `Todo.Api.UnitTests` project node in `Solution Explorer` and choose `Build`.
- Reload the VSCode window so that xUnit finds the tests:
  - Press `Ctrl + Shift + P` (`Cmd + Shift + P` on a Mac) and choose `Developer: Reload Window`.
- Click the Test Explorer icon ![](../images/test-explorer-view-icon.png).
- Finally, click on the ![](../images/run-tests-icon.png) icon to run the test, and verify everything is working as expected.

---

### Conclusion

This completes the introduction to xUnit, Moq and FluentAssertions in VSCode, where we have unit tested the TodoItemsController in the Todo.Api project.

Next, we will look at Integration Testing with Playwright in VSCode:

- You can close the VSCode instance with the `Utils` solution.
- Keep the VSCode instance with the `Todo` solution open (we will continue to use it in the next notebook).
- In the `notebooks` folder, open the file `integrationtesting.ipynb`.
- When the notebook opens in VSCode, click the text `Select Kernel` (top-right), and choose `Python Environments... => conda (Python 3.10.15) .conda/bin/python`.
- Now you can follow the instructions in the notebook.

---

### Additional Unit Testing Resources

This is a good introduction to Unit Testing with `xUnit` and `Moq`:

- A four part video series by Microsoft Learn about [Unit Testing](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-test-driven-and-scenario-based-testing) with an associated [GitHub Repository](https://github.com/skimedic/presentations/tree/master/UnitTesting):
  - [Part One](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-test-driven-and-scenario-based-testing) covers:
    - Creating an xUnit Test Project in Visual Studio and using the Test Explorer in Visual Studio.
    - The basics of [Test Driven Development (TDD)](https://en.wikipedia.org/wiki/Test-driven_development) and [Data Driven Testing (DDT)](https://en.wikipedia.org/wiki/Data-driven_testing) with [xUnit](https://xunit.net).
    - Using the [Arrange Act Assert (AAA) Pattern](https://learn.microsoft.com/en-us/dotnet/core/testing/unit-testing-best-practices#arranging-your-tests) in [Unit Testing Best Practices](https://learn.microsoft.com/en-us/dotnet/core/testing/unit-testing-best-practices).
    - The [Red Green Refactor Workflow](https://martinfowler.com/bliki/TestDrivenDevelopment.html) in [Test Driven Development (TDD)](https://en.wikipedia.org/wiki/Test-driven_development).
  - [Part Two](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-xunit) covers Unit Testing with [xUnit](https://xunit.net), including:
    - The attributes (decorators): `[Fact]`, `[Theory]`, `[InlineData]`, `[MemebrData]`, `[ClassData]`, `[Collection]`.
    - The assertion classes (and some of their methods): `Assert`, `Record`.
    - Test Fixtures:
      - Using the test class **Constructor** and `IDisposable` to setup and tear down a fixture for **each test** in a test class.
      - Class Fixtures with `IClassFixture<>`, `IDisposable` and [Dependency Injection](https://en.wikipedia.org/wiki/Dependency_injection) to setup and tear down a fixtue **shared by all tests** in a test class.
      - Note: **Collection Fixtures**, **Assembly Fixtures** and `IAsyncDisposable` aren't covered, but are explained in xUnit's documentation about [Shared Context between tests](https://xunit.net/docs/shared-context).
    - Using `ITestOutputHelper` via [Dependency Injection](https://en.wikipedia.org/wiki/Dependency_injection) and it's `WriteLine()` method to output text from tests.
  - [Part Three](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-moq-framework) covers Mocking with [Moq](https://github.com/devlooped/moq) and the concepts of [Mocks, Fakes, Stubs and Dummies](http://xunitpatterns.com/Mocks,%20Fakes,%20Stubs%20and%20Dummies.html), including:
    - Common assertion methods: `Assert.Same()`, `Assert.Equal()`, `Assert.Null()`, `Assert.Throws<>()`, `Record.Exception()`.
    - Common mock usage: `Mock<>()`, `mock.Object`, `mock.Setup().Returns()`, `mock.SetupSequqnce().Returns().Returns()`, `mock.Setup().Returns().Verifyable()`, `mock.Verify()`, `mock.Setup().Throws()`.
  - [Part Four](https://learn.microsoft.com/en-us/shows/visual-studio-toolbox/unit-testing-existing-code) explains how to add unit tests to existing code.